# FAQ recommendations

This Jupyter Notebook conducts an evaluation of various approaches to develop an LLM algorithm capable of recommending solutions based on an F.A.Q. database and the conversation context between an agent and a client.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 1 handling the data

## 1.1.0 Converting F.A.Q from .txt to .csv file

In [2]:
def get_questions_and_answers(faq_path: str):
    questions = []
    answers = []

    with open(faq_path, "r") as file:
        for line in file.readlines():
            if "Pergunta" in line:
                question = line.lstrip("Pergunta: ").rstrip("\n")
                questions.append(question)
            elif "Resposta" in line:
                answer = line.lstrip("Resposta: ").rstrip("\n")
                answers.append(answer)
    
    return (questions, answers)


def create_csv(questions: list, answers: list, destination_path: str, file_name: str):
    faq_dict = {
        "Question": questions,
        "Answer": answers
        }
    
    path = destination_path + file_name + ".csv"

    df = pd.DataFrame(data=faq_dict, dtype=object)
    df.to_csv(path_or_buf=path, sep=";", index=False)

In [3]:
questions, answers = get_questions_and_answers(faq_path="../data/raw/FAQ_example.txt")
create_csv(questions=questions, answers=answers, destination_path="../data/raw/", file_name="FAQ_example")

# 2 Testing LLMs techniques

## 2.1 Sentence Embeddings pretrained models

In [55]:
from sentence_transformers import SentenceTransformer, util
import spacy
import re


def get_greatest_similarities(model, dataframe, message):
    questions_embeddings = model.encode(dataframe.Question)
    message_embed = model.encode(get_clean_text(message))

    similarities = util.cos_sim(message_embed, questions_embeddings)[0].tolist()
    greatest_similarities_index = pd.Series(similarities).sort_values(ascending=False).index
    
    return greatest_similarities_index


def get_clean_text(text:str):
    nlp_model = spacy.load("pt_core_news_sm")

    text_tokens = nlp_model(text)

    clean_text = []

    for token in text_tokens:
        if not token.is_stop and not token.is_punct:
            clean_text.append(token.text)

    clean_text = " ".join(clean_text)

    return re.sub(r'[^a-zA-ZÀ-ÿ\s]', '', clean_text).lower()

In [56]:
df = pd.read_csv("../data/raw/FAQ_example.csv", sep=";")
df.head(3)

,Question,Answer
0,Quais serviços a empresa de telecomunicações o...,Nossa empresa oferece serviços de telefonia fi...
1,Como posso contratar os serviços da empresa?,"Para contratar nossos serviços, você pode entr..."
2,Quais são os planos disponíveis para a telefon...,Temos diversos planos com diferentes franquias...


In [58]:
message = "Como que eu faço para contratar um serviço melhor?"
message = get_clean_text(message)
print(message)

contratar serviço melhor


### 2.1.1 all-mpnet-base-v2 model

In [37]:
model = SentenceTransformer("all-mpnet-base-v2")

print(message)
df.iloc[get_greatest_similarities(model, df, message)].head(10)

contratar serviço melhor


,Question,Answer
1,Como posso contratar os serviços da empresa?,"Para contratar nossos serviços, você pode entr..."
17,Como posso solicitar a mudança de endereço dos...,Entre em contato com nosso atendimento para so...
18,É possível fazer a rescisão dos serviços a qua...,"Sim, você pode solicitar a rescisão dos serviç..."
23,A empresa oferece serviço de fibra óptica?,"Sim, nossa empresa oferece serviço de internet..."
8,osso fazer a contratação de serviços adicionai...,"Sim, você pode personalizar seu pacote de TV p..."
3,A empresa oferece planos empresariais?,"Sim, temos planos especiais para empresas, com..."
24,Como posso agendar a visita de um técnico para...,Você pode agendar a visita de um técnico entra...
10,É possível fazer upgrade ou downgrade nos plan...,"Sim, você pode fazer o upgrade ou downgrade do..."
15,"É possível contratar serviços adicionais, como...","Sim, oferecemos opções de streaming em parceri..."
11,A empresa oferece descontos para pacotes combi...,"Sim, oferecemos descontos especiais para clien..."


### 2.1.2 all-MiniLM-L6-v2 model

In [43]:
model = SentenceTransformer("all-MiniLM-L6-v2")

print(message)
df.iloc[get_greatest_similarities(model, df, message)].head(10)

contratar serviço melhor


,Question,Answer
17,Como posso solicitar a mudança de endereço dos...,Entre em contato com nosso atendimento para so...
1,Como posso contratar os serviços da empresa?,"Para contratar nossos serviços, você pode entr..."
18,É possível fazer a rescisão dos serviços a qua...,"Sim, você pode solicitar a rescisão dos serviç..."
8,osso fazer a contratação de serviços adicionai...,"Sim, você pode personalizar seu pacote de TV p..."
23,A empresa oferece serviço de fibra óptica?,"Sim, nossa empresa oferece serviço de internet..."
15,"É possível contratar serviços adicionais, como...","Sim, oferecemos opções de streaming em parceri..."
10,É possível fazer upgrade ou downgrade nos plan...,"Sim, você pode fazer o upgrade ou downgrade do..."
24,Como posso agendar a visita de um técnico para...,Você pode agendar a visita de um técnico entra...
7,A empresa oferece suporte técnico?,"Sim, nossa empresa oferece suporte técnico esp..."
16,A empresa oferece atendimento em outros idioma...,"No momento, nosso atendimento é realizado apen..."


### 2.1.3 msmarco-distilbert-base-v4

In [42]:
model = SentenceTransformer("msmarco-distilbert-base-v4")

print(message)
df.iloc[get_greatest_similarities(model, df, message)].head(10)

contratar serviço melhor


,Question,Answer
1,Como posso contratar os serviços da empresa?,"Para contratar nossos serviços, você pode entr..."
17,Como posso solicitar a mudança de endereço dos...,Entre em contato com nosso atendimento para so...
15,"É possível contratar serviços adicionais, como...","Sim, oferecemos opções de streaming em parceri..."
21,A empresa disponibiliza serviços de telefonia ...,"Sim, oferecemos opções para ligações internaci..."
18,É possível fazer a rescisão dos serviços a qua...,"Sim, você pode solicitar a rescisão dos serviç..."
26,A empresa oferece serviços de telefonia VoIP?,"Sim, oferecemos serviços de telefonia VoIP par..."
8,osso fazer a contratação de serviços adicionai...,"Sim, você pode personalizar seu pacote de TV p..."
23,A empresa oferece serviço de fibra óptica?,"Sim, nossa empresa oferece serviço de internet..."
0,Quais serviços a empresa de telecomunicações o...,Nossa empresa oferece serviços de telefonia fi...
24,Como posso agendar a visita de um técnico para...,Você pode agendar a visita de um técnico entra...


### 2.1.4 bert-large-portuguese-cased (Finetuned model)

In [78]:
from sentence_transformers import models

model = models.Transformer("../models/bert-large-portuguese-cased")
pooling_model = models.Pooling(model.get_word_embedding_dimension())

model = SentenceTransformer(modules=[model, pooling_model])

print(message)
df.iloc[get_greatest_similarities(model, df, message)].head(10)

contratar serviço melhor


,Question,Answer
1,Como posso contratar os serviços da empresa?,"Para contratar nossos serviços, você pode entr..."
7,A empresa oferece suporte técnico?,"Sim, nossa empresa oferece suporte técnico esp..."
21,A empresa disponibiliza serviços de telefonia ...,"Sim, oferecemos opções para ligações internaci..."
11,A empresa oferece descontos para pacotes combi...,"Sim, oferecemos descontos especiais para clien..."
0,Quais serviços a empresa de telecomunicações o...,Nossa empresa oferece serviços de telefonia fi...
3,A empresa oferece planos empresariais?,"Sim, temos planos especiais para empresas, com..."
8,osso fazer a contratação de serviços adicionai...,"Sim, você pode personalizar seu pacote de TV p..."
23,A empresa oferece serviço de fibra óptica?,"Sim, nossa empresa oferece serviço de internet..."
29,A empresa disponibiliza pacotes de internet co...,"Sim, temos opções de planos com tráfego ilimit..."
17,Como posso solicitar a mudança de endereço dos...,Entre em contato com nosso atendimento para so...


# 3 Preparing to Deploy

In [82]:
from sentence_transformers import models

word_embedding_model = models.Transformer("../models/bert-large-portuguese-cased")
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())

embedding_model = SentenceTransformer(modules=[model, pooling_model])

In [83]:
nlp_model = spacy.load("pt_core_news_sm")

In [84]:
df = pd.read_csv("../data/raw/FAQ_example.csv", sep=";")
df.head(3)

,Question,Answer
0,Quais serviços a empresa de telecomunicações o...,Nossa empresa oferece serviços de telefonia fi...
1,Como posso contratar os serviços da empresa?,"Para contratar nossos serviços, você pode entr..."
2,Quais são os planos disponíveis para a telefon...,Temos diversos planos com diferentes franquias...


## 3.1 Preparing Data

### 3.1.1 Creating Embeddings for each question and exporting as CSV file

In [85]:
def get_sentence_embeddings(model, series):
    """This function removes all 'stop words' for each sentence in a pandas.Series
    and then creates a embed for the sentence without the stop words."""

    for i, text in enumerate(series):
        embed = model.encode(get_clean_text(text))
        
        if i == 0:
            text_embeddings = pd.DataFrame({f"{i}": embed})
        else:
            text_embeddings[f"{i}"] = embed

    return text_embeddings

In [86]:
df_embeddings = get_sentence_embeddings(embeddings_model, df.Question)
df_embeddings.head()

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,0.123091,0.385130,-0.042114,-0.073823,-0.241438,0.120406,0.030073,-0.049525,0.308561,0.618427,...,0.060034,0.496666,-0.259602,0.414070,-0.172650,-0.421446,0.510475,-0.940662,-0.530580,-0.047649
1,0.482359,0.556570,-0.708709,-0.043453,0.226264,0.229228,-0.210801,0.469060,0.189250,-0.240432,...,0.091829,0.368335,-0.595064,0.295027,0.209844,-0.089121,0.255039,0.395473,0.183490,0.530826
2,0.439281,0.431969,0.750868,0.382758,-0.033281,0.214516,0.445881,0.223533,0.543519,0.036400,...,1.010078,1.045076,0.711472,0.830189,0.741978,0.305951,0.655227,0.421352,0.456598,0.316111
3,0.440510,0.174288,-0.233712,-0.026013,0.430831,0.165883,0.317461,0.420585,-0.107572,0.264198,...,0.080298,0.140680,0.582493,0.169119,-0.249130,0.463767,0.335840,0.129245,0.256666,0.610861
4,0.848422,0.573981,0.950692,1.247029,0.724088,0.301497,0.368689,0.606544,0.565717,0.440187,...,0.268479,0.446494,0.711747,0.766761,-0.064110,0.493369,0.775443,0.066883,0.670383,0.597269


In [87]:
df_embeddings.to_csv(path_or_buf="../data/processed/Embeddings.csv", sep=";", index=False)

## 3.2 Exporting the model with the best performance

### 3.2.1 Embeddings model

In [88]:
embeddings_model.save('../models/embeddings_model')

### 3.2.2 NLP Model

In [89]:
nlp_model.to_disk('../models/nlp_model')